# Setup

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
print("sklearn version: ", sklearn.__version__)
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
print("TF version: ", tf.__version__)
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# GPU test
print("GPU installed: ",tf.test.is_built_with_gpu_support())

# To prevent "CUDNN_STATUS_ALLOC_FAILED" error with GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    
# Common imports
import numpy as np
import os
import csv
import cv2 as cv
if IS_COLAB:
    !pip install tensorflow_addons --quiet
    from google.colab import drive
import tensorflow_addons as tfa
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import gc
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
from enum import Enum
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, AveragePooling2D, MaxPooling2D, Dropout
from keras.initializers import HeUniform, LecunUniform

from datetime import datetime
from functools import partial
from tensorflow.python.keras.utils.data_utils import Sequence

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")    

!nvidia-smi -q -i 0 | grep "Product Name"

sklearn version:  1.0.2
TF version:  2.8.2
GPU installed:  True
1 Physical GPUs, 1 Logical GPUs
     |████████████████████████████████| 1.1 MB 6.3 MB/s 
    Product Name                          : Tesla P100-PCIE-16GB


# Get the data

In [ ]:
if IS_COLAB:
    drive.mount("/content/drive")
    location = "drive/MyDrive/Colab Notebooks/"
else:
    location = "./"
# 작성자 전민재, hsh
def load_Emist(exsitNumpy=False, needTranspose=True):
    if exsitNumpy == False:
        # "./emnist-byclass-test.csv"
        # train-set
        # "공용/datasets/Emnist/" 위치에 csv 저장
        csv_train_data_file = open(
            location + "datasets/Emnist/train_data.csv",
            "r",
            encoding="ms932",
            errors="",
            newline="",
        )  # 리스트 형식
        csv_train_label_file = open(
            location + "datasets/Emnist/train_label.csv",
            "r",
            encoding="ms932",
            errors="",
            newline="",
        )  # 리스트 형식
        f_train = csv.reader(
            csv_train_data_file,
            delimiter=",",
            doublequote=True,
            lineterminator="\r\n",
            quotechar='"',
            skipinitialspace=True,
        )
        l_train = csv.reader(
            csv_train_label_file,
            delimiter=",",
            doublequote=True,
            lineterminator="\r\n",
            quotechar='"',
            skipinitialspace=True,
        )
        # valid-set
        csv_valid_data_file = open(
            location + "datasets/Emnist/valid_data.csv",
            "r",
            encoding="ms932",
            errors="",
            newline="",
        )  # 리스트 형식
        csv_valid_label_file = open(
            location + "datasets/Emnist/valid_label.csv",
            "r",
            encoding="ms932",
            errors="",
            newline="",
        )  # 리스트 형식
        f_valid = csv.reader(
            csv_valid_data_file,
            delimiter=",",
            doublequote=True,
            lineterminator="\r\n",
            quotechar='"',
            skipinitialspace=True,
        )
        l_valid = csv.reader(
            csv_valid_label_file,
            delimiter=",",
            doublequote=True,
            lineterminator="\r\n",
            quotechar='"',
            skipinitialspace=True,
        )
        X_train = []
        y_train = []
        X_valid = []
        y_valid = []
        for i, row in enumerate(f_train):
            # 행마다 int로 형변환
            for idx, char in enumerate(row):
                row[idx] = int(char)
            # train
            # data 추가
            X_train.append(row)
        for i, row in enumerate(f_valid):
            # 행마다 int로 형변환
            for idx, char in enumerate(row):
                row[idx] = int(char)
            # valid
            # data 추가
            X_valid.append(row)
        for i, row in enumerate(l_train):
            # train
            # label 추가
            y_train.append(int(row[0]))
        for i, row in enumerate(l_valid):
            # valid
            # label 추가
            y_valid.append(int(row[0]))
        # train
        X_train = np.array(X_train, dtype=np.uint8)
        X_train = X_train.reshape(-1, 28, 28)
        # valid
        X_valid = np.array(X_valid, dtype=np.uint8)
        X_valid = X_valid.reshape(-1, 28, 28)
        csv_train_data_file.close()
        csv_train_label_file.close()
        csv_valid_data_file.close()
        csv_valid_label_file.close()
        # kaggle dataset이 시계반대방향으로 90도 회전 되있고 상하 반전 되어있음
        def rotate_90(m):
            N = len(m)
            ret = [[0] * N for _ in range(N)]

            for r in range(N):
                for c in range(N):
                    ret[c][N - 1 - r] = m[r][c]
            return np.array(ret, dtype=np.uint8)

        def vreflect(m):
            N = len(m)
            ret = [[0] * N for _ in range(N)]

            for r in range(N):
                for c in range(N):
                    ret[r][c] = m[r][N - 1 - c]
            return np.array(ret, dtype=np.uint8)

        if needTranspose == True:
            # train
            # 회전
            for idx, i in enumerate(X_train):
                X_train[idx] = rotate_90(i)
            # 상하반전
            for idx, i in enumerate(X_train):
                X_train[idx] = vreflect(i)
            # valid
            # 회전
            for idx, i in enumerate(X_valid):
                X_valid[idx] = rotate_90(i)
            # 상하반전
            for idx, i in enumerate(X_valid):
                X_valid[idx] = vreflect(i)
        # 저장용
        # train
        # (533917,28,28)
        np.save(location + "datasets/Emnist/X_train", np.array(X_train, dtype=np.uint8))
        np.save(location + "datasets/Emnist/y_train", np.array(y_train, dtype=np.uint8))
        # valid
        np.save(location + "datasets/Emnist/X_valid", np.array(X_valid, dtype=np.uint8))
        np.save(location + "datasets/Emnist/y_valid", np.array(y_valid, dtype=np.uint8))
    X_train = np.load(location + "datasets/Emnist/X_train.npy")
    y_train = np.load(location + "datasets/Emnist/y_train.npy")
    X_valid = np.load(location + "datasets/Emnist/X_valid.npy")
    y_valid = np.load(location + "datasets/Emnist/y_valid.npy")
    return X_train, y_train, X_valid, y_valid

Mounted at /content/drive


In [ ]:
# 황성현
# npy 파일이 존재할 경우 exsitNumpy = True, 없으면 False
X_train, y_train, X_valid, y_valid = load_Emist(exsitNumpy=True)
X_train = tf.reshape(X_train, [-1, 28, 28, 1])
X_valid = tf.reshape(X_valid, [-1, 28, 28, 1])
X_train_resize = np.uint8(tf.image.resize(X_train, [32, 32]))
X_valid_resize = np.uint8(tf.image.resize(X_valid, [32, 32]))

# Train the Model

In [ ]:
model = tf.keras.applications.MobileNetV2(
    input_shape=(32, 32, 1),
    alpha=1.0,
    include_top=True,
    weights=None,
    input_tensor=None,
    pooling=None,
    classes=62,
    classifier_activation="softmax",
)

In [ ]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/mobailnetv2"
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir = logs, histogram_freq = 1, profile_batch = "500, 520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath = location + "weights/mobilenetv2_best.h5", save_best_only = True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath = location + "weights/mobilenetv2_last.h5", save_best_only = False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
history = model.fit(
    x=X_train_resize,
    y=y_train,
    validation_data=(X_valid_resize, y_valid),
    epochs = 100,
    callbacks = [tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
16685/16685 [==============================] - 392s 23ms/step - loss: 0.6970 - accuracy: 0.7761 - val_loss: 0.5463 - val_accuracy: 0.8324
Epoch 2/100
16685/16685 [==============================] - 387s 23ms/step - loss: 0.4560 - accuracy: 0.8401 - val_loss: 0.4556 - val_accuracy: 0.8422
Epoch 3/100
16685/16685 [==============================] - 377s 23ms/step - loss: 0.4167 - accuracy: 0.8510 - val_loss: 0.4121 - val_accuracy: 0.8460
Epoch 4/100
16685/16685 [==============================] - 381s 23ms/step - loss: 0.3964 - accuracy: 0.8566 - val_loss: 0.4037 - val_accuracy: 0.8529
Epoch 5/100
16685/16685 [==============================] - 382s 23ms/step - loss: 0.3834 - accuracy: 0.8602 - val_loss: 0.3835 - val_accuracy: 0.8613
Epoch 6/100
16685/16685 [==============================] - 374s 22ms/step - loss: 0.3738 - accuracy: 0.8629 - val_loss: 0.3827 - val_accuracy: 0.8620
Epoch 7/100
16685/16685 [==============================] - 373s 22ms/step - loss: 0.3668 - accuracy:

# Change Optimizer

In [ ]:
model = tf.keras.applications.MobileNetV2(
    input_shape=(32, 32, 1),
    alpha=1.0,
    include_top=True,
    weights=None,
    input_tensor=None,
    pooling=None,
    classes=62,
    classifier_activation="softmax",
)

In [ ]:
model.compile(
    optimizer="nadam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/mobilenetv2_nadam"
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir = logs, histogram_freq = 1, profile_batch = "500, 520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath = location + "weights/mobilenetv2_nadam_best.h5", save_best_only = True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath = location + "weights/mobilenetv2_nadam_last.h5", save_best_only = False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
history = model.fit(
    x=X_train_resize,
    y=y_train,
    validation_data=(X_valid_resize, y_valid),
    epochs = 100,
    callbacks = [tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
16685/16685 [==============================] - 687s 40ms/step - loss: 0.6858 - accuracy: 0.7782 - val_loss: 0.6135 - val_accuracy: 0.8205
Epoch 2/100
16685/16685 [==============================] - 606s 36ms/step - loss: 0.4518 - accuracy: 0.8408 - val_loss: 0.4401 - val_accuracy: 0.8473
Epoch 3/100
16685/16685 [==============================] - 583s 35ms/step - loss: 0.4139 - accuracy: 0.8517 - val_loss: 0.4253 - val_accuracy: 0.8450
Epoch 4/100
16685/16685 [==============================] - 592s 35ms/step - loss: 0.3956 - accuracy: 0.8564 - val_loss: 0.3937 - val_accuracy: 0.8561
Epoch 5/100
16685/16685 [==============================] - 594s 36ms/step - loss: 0.3829 - accuracy: 0.8600 - val_loss: 0.3935 - val_accuracy: 0.8575
Epoch 6/100
16685/16685 [==============================] - 619s 37ms/step - loss: 0.3738 - accuracy: 0.8623 - val_loss: 0.3784 - val_accuracy: 0.8627
Epoch 7/100
16685/16685 [==============================] - 601s 36ms/step - loss: 0.3668 - accuracy:

# Change pooling parameter

In [ ]:
model = tf.keras.applications.MobileNetV2(
    input_shape=(32, 32, 1),
    alpha=1.0,
    include_top=True,
    weights=None,
    input_tensor=None,
    pooling="avg",
    classes=62,
    classifier_activation="softmax",
)

In [ ]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/" + datetime.now().strftime("mobilenetv2_avg_pooling%Y%m%d-%H%M%S")
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir = logs, histogram_freq = 1, profile_batch = "500, 520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath = location + "weights/mobilenetv2_avg_pooling_best.h5", save_best_only = True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath = location + "weights/mobilenetv2_avg_pooling_last.h5", save_best_only = False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
history = model.fit(
    x=X_train_resize,
    y=y_train,
    validation_data=(X_valid_resize, y_valid),
    epochs = 100,
    callbacks = [tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
16685/16685 [==============================] - 394s 23ms/step - loss: 0.7028 - accuracy: 0.7753 - val_loss: 0.6131 - val_accuracy: 0.8093
Epoch 2/100
16685/16685 [==============================] - 380s 23ms/step - loss: 0.4553 - accuracy: 0.8397 - val_loss: 0.4670 - val_accuracy: 0.8398
Epoch 3/100
16685/16685 [==============================] - 376s 23ms/step - loss: 0.4160 - accuracy: 0.8510 - val_loss: 0.4033 - val_accuracy: 0.8515
Epoch 4/100
16685/16685 [==============================] - 383s 23ms/step - loss: 0.3994 - accuracy: 0.8554 - val_loss: 0.3841 - val_accuracy: 0.8601
Epoch 5/100
16685/16685 [==============================] - 382s 23ms/step - loss: 0.3879 - accuracy: 0.8580 - val_loss: 0.3785 - val_accuracy: 0.8617
Epoch 6/100
16685/16685 [==============================] - 374s 22ms/step - loss: 0.3765 - accuracy: 0.8614 - val_loss: 0.3822 - val_accuracy: 0.8610
Epoch 7/100
16685/16685 [==============================] - 384s 23ms/step - loss: 0.3715 - accuracy:

In [ ]:
model = tf.keras.applications.MobileNetV2(
    input_shape=(32, 32, 1),
    alpha=1.0,
    include_top=True,
    weights=None,
    input_tensor=None,
    pooling="max",
    classes=62,
    classifier_activation="softmax",
)

In [ ]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/" + datetime.now().strftime("mobilenetv2_max_pooling%Y%m%d-%H%M%S")
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir = logs, histogram_freq = 1, profile_batch = "500, 520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath = location + "weights/mobilenetv2_max_pooling_best.h5", save_best_only = True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath = location + "weights/mobilenetv2_max_pooling_last.h5", save_best_only = False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
history = model.fit(
    x=X_train_resize,
    y=y_train,
    validation_data=(X_valid_resize, y_valid),
    epochs = 100,
    callbacks = [tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
16685/16685 [==============================] - 396s 23ms/step - loss: 0.6925 - accuracy: 0.7776 - val_loss: 0.6652 - val_accuracy: 0.7890
Epoch 2/100
16685/16685 [==============================] - 375s 22ms/step - loss: 0.4521 - accuracy: 0.8407 - val_loss: 0.4764 - val_accuracy: 0.8381
Epoch 3/100
16685/16685 [==============================] - 385s 23ms/step - loss: 0.4139 - accuracy: 0.8515 - val_loss: 0.4049 - val_accuracy: 0.8525
Epoch 4/100
16685/16685 [==============================] - 383s 23ms/step - loss: 0.3954 - accuracy: 0.8565 - val_loss: 0.3866 - val_accuracy: 0.8549
Epoch 5/100
16685/16685 [==============================] - 375s 22ms/step - loss: 0.3832 - accuracy: 0.8597 - val_loss: 0.3874 - val_accuracy: 0.8599
Epoch 6/100
16685/16685 [==============================] - 375s 22ms/step - loss: 0.3754 - accuracy: 0.8620 - val_loss: 0.3972 - val_accuracy: 0.8548
Epoch 7/100
16685/16685 [==============================] - 370s 22ms/step - loss: 0.3698 - accuracy:

# Padding instead of resize

In [ ]:
X_train, y_train, X_valid, y_valid = load_Emist(exsitNumpy=True)
X_train = tf.reshape(X_train, [-1, 28, 28, 1])
X_valid = tf.reshape(X_valid, [-1, 28, 28, 1])
X_train_pad = np.uint8(tf.image.pad_to_bounding_box(X_train, 2, 2, 32, 32))
X_valid_pad = np.uint8(tf.image.pad_to_bounding_box(X_valid, 2, 2, 32, 32))

In [ ]:
def mobilenetv2_model():
    return tf.keras.applications.MobileNetV2(
        input_shape=(32, 32, 1),
        alpha=1.0,
        include_top=True,
        weights=None,
        input_tensor=None,
        pooling=None,
        classes=62,
        classifier_activation="softmax",
)

In [ ]:
model = mobilenetv2_model()
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/mobilenet_v2_pad"
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir = logs, histogram_freq = 1, profile_batch = "500, 520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath = location + "weights/mobilenetv2_pad_best.h5", save_best_only = True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath = location + "weights/mobilenetv2_pad_last.h5", save_best_only = False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
history = model.fit(
    x=X_train_pad,
    y=y_train,
    validation_data=(X_valid_pad, y_valid),
    epochs = 100,
    callbacks = [tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
16685/16685 [==============================] - 346s 19ms/step - loss: 0.7122 - accuracy: 0.7725 - val_loss: 0.5862 - val_accuracy: 0.8244
Epoch 2/100
16685/16685 [==============================] - 316s 19ms/step - loss: 0.4580 - accuracy: 0.8390 - val_loss: 0.4569 - val_accuracy: 0.8434
Epoch 3/100
16685/16685 [==============================] - 324s 19ms/step - loss: 0.4197 - accuracy: 0.8494 - val_loss: 0.4313 - val_accuracy: 0.8464
Epoch 4/100
16685/16685 [==============================] - 314s 19ms/step - loss: 0.4016 - accuracy: 0.8551 - val_loss: 0.3923 - val_accuracy: 0.8562
Epoch 5/100
16685/16685 [==============================] - 314s 19ms/step - loss: 0.3906 - accuracy: 0.8581 - val_loss: 0.3873 - val_accuracy: 0.8595
Epoch 6/100
16685/16685 [==============================] - 311s 19ms/step - loss: 0.3812 - accuracy: 0.8601 - val_loss: 0.3930 - val_accuracy: 0.8594
Epoch 7/100
16685/16685 [==============================] - 325s 19ms/step - loss: 0.3752 - accuracy:

# Change Batch Size

In [ ]:
def mobilenetv2_model():
    return tf.keras.applications.MobileNetV2(
        input_shape=(32, 32, 1),
        alpha=1.0,
        include_top=True,
        weights=None,
        input_tensor=None,
        pooling=None,
        classes=62,
        classifier_activation="softmax",
)

In [ ]:
model = mobilenetv2_model()
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/mobilenetv2_batch_16"
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir = logs, histogram_freq = 1, profile_batch = "500, 520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath = location + "weights/mobilenetv2_batch_16_best.h5", save_best_only = True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath = location + "weights/mobilenetv2_batch_16_last.h5", save_best_only = False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
history = model.fit(
    x=X_train_pad,
    y=y_train,
    batch_size=16,
    validation_data=(X_valid_pad, y_valid),
    epochs = 100,
    callbacks = [tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
33370/33370 [==============================] - 681s 20ms/step - loss: 0.7608 - accuracy: 0.7609 - val_loss: 0.5184 - val_accuracy: 0.8225
Epoch 2/100
33370/33370 [==============================] - 679s 20ms/step - loss: 0.4797 - accuracy: 0.8347 - val_loss: 0.4407 - val_accuracy: 0.8457
Epoch 3/100
33370/33370 [==============================] - 678s 20ms/step - loss: 0.4425 - accuracy: 0.8447 - val_loss: 0.4177 - val_accuracy: 0.8520
Epoch 4/100
33370/33370 [==============================] - 658s 20ms/step - loss: 0.4209 - accuracy: 0.8509 - val_loss: 0.4028 - val_accuracy: 0.8579
Epoch 5/100
33370/33370 [==============================] - 661s 20ms/step - loss: 0.4076 - accuracy: 0.8543 - val_loss: 0.3983 - val_accuracy: 0.8568
Epoch 6/100
33370/33370 [==============================] - 689s 21ms/step - loss: 0.3967 - accuracy: 0.8575 - val_loss: 0.3973 - val_accuracy: 0.8596
Epoch 7/100
33370/33370 [==============================] - 662s 20ms/step - loss: 0.3894 - accuracy:

In [ ]:
model = mobilenetv2_model()
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/mobilenetv2_batch_64"
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir = logs, histogram_freq = 1, profile_batch = "500, 520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath = location + "weights/mobilenetv2_batch_64_best.h5", save_best_only = True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath = location + "weights/mobilenetv2_batch_64_last.h5", save_best_only = False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
history = model.fit(
    x=X_train_pad,
    y=y_train,
    batch_size=64,
    validation_data=(X_valid_pad, y_valid),
    epochs = 100,
    callbacks = [tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
8343/8343 [==============================] - 208s 23ms/step - loss: 0.6966 - accuracy: 0.7767 - val_loss: 1.1372 - val_accuracy: 0.7549
Epoch 2/100
8343/8343 [==============================] - 190s 23ms/step - loss: 0.4550 - accuracy: 0.8395 - val_loss: 0.5251 - val_accuracy: 0.8366
Epoch 3/100
8343/8343 [==============================] - 185s 22ms/step - loss: 0.4139 - accuracy: 0.8505 - val_loss: 0.4454 - val_accuracy: 0.8452
Epoch 4/100
8343/8343 [==============================] - 185s 22ms/step - loss: 0.3906 - accuracy: 0.8577 - val_loss: 0.4162 - val_accuracy: 0.8492
Epoch 5/100
8343/8343 [==============================] - 187s 22ms/step - loss: 0.3802 - accuracy: 0.8603 - val_loss: 0.4001 - val_accuracy: 0.8567
Epoch 6/100
8343/8343 [==============================] - 184s 22ms/step - loss: 0.3674 - accuracy: 0.8638 - val_loss: 0.4074 - val_accuracy: 0.8551
Epoch 7/100
8343/8343 [==============================] - 184s 22ms/step - loss: 0.3604 - accuracy: 0.8656 - val_

In [ ]:
model = mobilenetv2_model()
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/mobilenetv2_batch_128"
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir = logs, histogram_freq = 1, profile_batch = "500, 520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath = location + "weights/mobilenetv2_batch_128_best.h5", save_best_only = True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath = location + "weights/mobilenetv2_batch_128_last.h5", save_best_only = False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
history = model.fit(
    x=X_train_pad,
    y=y_train,
    batch_size=128,
    validation_data=(X_valid_pad, y_valid),
    epochs = 100,
    callbacks = [tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
4172/4172 [==============================] - 127s 29ms/step - loss: 0.6797 - accuracy: 0.7788 - val_loss: 3.5945 - val_accuracy: 0.0490
Epoch 2/100
4172/4172 [==============================] - 122s 29ms/step - loss: 0.4513 - accuracy: 0.8397 - val_loss: 1.4455 - val_accuracy: 0.7223
Epoch 3/100
4172/4172 [==============================] - 111s 27ms/step - loss: 0.4131 - accuracy: 0.8508 - val_loss: 0.8447 - val_accuracy: 0.7805
Epoch 4/100
4172/4172 [==============================] - 119s 29ms/step - loss: 0.3890 - accuracy: 0.8573 - val_loss: 0.4835 - val_accuracy: 0.8434
Epoch 5/100
4172/4172 [==============================] - 117s 28ms/step - loss: 0.3738 - accuracy: 0.8617 - val_loss: 0.4602 - val_accuracy: 0.8485
Epoch 6/100
4172/4172 [==============================] - 108s 26ms/step - loss: 0.3625 - accuracy: 0.8652 - val_loss: 0.4353 - val_accuracy: 0.8497
Epoch 7/100
4172/4172 [==============================] - 108s 26ms/step - loss: 0.3543 - accuracy: 0.8673 - val_

In [9]:
model = mobilenetv2_model()
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/mobilenetv2_batch_256"
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir = logs, histogram_freq = 1, profile_batch = "500, 520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath = location + "weights/mobilenetv2_batch_256_best.h5", save_best_only = True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath = location + "weights/mobilenetv2_batch_256_last.h5", save_best_only = False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
history = model.fit(
    x=X_train_pad,
    y=y_train,
    batch_size=256,
    validation_data=(X_valid_pad, y_valid),
    epochs = 100,
    callbacks = [tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
2086/2086 [==============================] - 86s 39ms/step - loss: 0.7043 - accuracy: 0.7741 - val_loss: 4.0845 - val_accuracy: 0.0489
Epoch 2/100
2086/2086 [==============================] - 80s 38ms/step - loss: 0.4407 - accuracy: 0.8433 - val_loss: 3.8499 - val_accuracy: 0.0489
Epoch 3/100
2086/2086 [==============================] - 79s 38ms/step - loss: 0.4102 - accuracy: 0.8518 - val_loss: 0.6314 - val_accuracy: 0.8142
Epoch 4/100
2086/2086 [==============================] - 79s 38ms/step - loss: 0.3898 - accuracy: 0.8574 - val_loss: 0.8383 - val_accuracy: 0.7978
Epoch 5/100
2086/2086 [==============================] - 79s 38ms/step - loss: 0.3789 - accuracy: 0.8604 - val_loss: 0.8416 - val_accuracy: 0.7821
Epoch 6/100
2086/2086 [==============================] - 81s 39ms/step - loss: 0.3678 - accuracy: 0.8633 - val_loss: 0.6138 - val_accuracy: 0.8195
Epoch 7/100
2086/2086 [==============================] - 80s 38ms/step - loss: 0.3556 - accuracy: 0.8671 - val_loss: 0

In [10]:
model = mobilenetv2_model()
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
logs = location + "logs/mobilenetv2_batch_512"
tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir = logs, histogram_freq = 1, profile_batch = "500, 520"
)
check_best_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath = location + "weights/mobilenetv2_batch_512_best.h5", save_best_only = True
)
check_last_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath = location + "weights/mobilenetv2_batch_512_last.h5", save_best_only = False
)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
history = model.fit(
    x=X_train_pad,
    y=y_train,
    batch_size=512,
    validation_data=(X_valid_pad, y_valid),
    epochs = 100,
    callbacks = [tboard_callback, check_best_cb, check_last_cb, earlystop_cb],
)

Epoch 1/100
1043/1043 [==============================] - 74s 67ms/step - loss: 0.7678 - accuracy: 0.7561 - val_loss: 3.9770 - val_accuracy: 0.0485
Epoch 2/100
1043/1043 [==============================] - 68s 65ms/step - loss: 0.4398 - accuracy: 0.8424 - val_loss: 3.9862 - val_accuracy: 0.0485
Epoch 3/100
1043/1043 [==============================] - 69s 66ms/step - loss: 0.4065 - accuracy: 0.8522 - val_loss: 3.9681 - val_accuracy: 0.0489
Epoch 4/100
1043/1043 [==============================] - 69s 66ms/step - loss: 0.3891 - accuracy: 0.8575 - val_loss: 3.7455 - val_accuracy: 0.1197
Epoch 5/100
1043/1043 [==============================] - 69s 66ms/step - loss: 0.3771 - accuracy: 0.8605 - val_loss: 0.6958 - val_accuracy: 0.8243
Epoch 6/100
1043/1043 [==============================] - 67s 65ms/step - loss: 0.3670 - accuracy: 0.8637 - val_loss: 0.8168 - val_accuracy: 0.7549
Epoch 7/100
1043/1043 [==============================] - 68s 65ms/step - loss: 0.3591 - accuracy: 0.8658 - val_loss: 1